In [3]:
!pip install pandas XlsxWriter

In [4]:
import os
import shutil
from collections import defaultdict
import pandas as pd

# Function to map directory structure
def map_directory(root_folder):
    paths = []
    file_data = []

    for dirpath, dirnames, filenames in os.walk(root_folder):
        relative_path = os.path.relpath(dirpath, root_folder)
        parts = relative_path.split(os.sep)
        
        for filename in filenames:
            row = [root_folder] + parts
            paths.append(row)
            full_path = os.path.join(dirpath, filename)
            file_data.append([filename, full_path])

    max_depth = max(len(row) for row in paths)

    for row in paths:
        row.extend([''] * (max_depth - len(row)))

    for i, row in enumerate(paths):
        row.append(file_data[i][0])
        row.append(file_data[i][1])

    columns_structure = ['Root Directory'] + [f'Level {i}' for i in range(1, max_depth)] + ['File Name', 'Original Path']
    df_structure = pd.DataFrame(paths, columns=columns_structure)

    return df_structure

# Function to copy and rename files
def copy_and_rename_files(source_dir, target_dir):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    
    file_counter = defaultdict(int)
    file_info = []

    for root, _, files in os.walk(source_dir):
        for file in files:
            file_ext = os.path.splitext(file)[1]
            file_name = os.path.splitext(file)[0]
            file_counter[file_name] += 1
            new_file_name = f"{file_name}_{file_counter[file_name]}{file_ext}"
            
            source_file_path = os.path.join(root, file)
            target_file_path = os.path.join(target_dir, new_file_name)
            
            shutil.copy2(source_file_path, target_file_path)
            file_info.append((file, target_file_path, file_ext, source_file_path))
    
    return file_info

# Function to organize files by extension
def organize_files_by_extension(target_dir):
    file_locations = []

    for root, _, files in os.walk(target_dir):
        for file in files:
            file_ext = os.path.splitext(file)[1][1:]
            if file_ext:
                ext_folder_path = os.path.join(target_dir, file_ext)
                if not os.path.exists(ext_folder_path):
                    os.makedirs(ext_folder_path)
                
                source_file_path = os.path.join(root, file)
                target_file_path = os.path.join(ext_folder_path, file)
                
                shutil.move(source_file_path, target_file_path)
                file_locations.append((source_file_path, target_file_path, file_ext))
    
    return file_locations

# Function to create the final Excel report
def create_combined_report(df_structure, file_info, file_locations, excel_path, base_folder):
    data = []
    file_location_dict = {new_path: (old_name, ext, orig_path) for old_name, new_path, ext, orig_path in file_info}
    
    for new_path, final_path, ext in file_locations:
        old_name, original_ext, orig_path = file_location_dict.get(new_path, (None, None, None))
        if orig_path is not None:
            folder_name = os.path.basename(os.path.dirname(final_path))
            data.append([orig_path, os.path.basename(new_path), base_folder, folder_name])
    
    df_new_files = pd.DataFrame(data, columns=["Original Path", "Renamed File Name", "New Folder Assignment", "Folder"])

    # Merge the directory structure DataFrame with the new files DataFrame
    df_merged = pd.merge(df_structure, df_new_files, on='Original Path', how='left')

    # Drop the unnecessary columns
    df_merged.drop(columns=["Original Path"], inplace=True)
    
    # Save the merged DataFrame to the final Excel file
    df_merged.to_excel(excel_path, index=False)

def main():
    source_dir = 'TestDirectory'   #<-- name for the directory to map and copy/categorize.
    target_dir = 'cleanedDir' #<-- name for new directory
    final_excel_path = 'CombinedFileTracker22.xlsx' #<-- name for excel file
    base_folder = target_dir

    # Step 1: Map the directory structure
    df_structure = map_directory(source_dir)
    
    # Step 2: Copy, rename, and organize files
    file_info = copy_and_rename_files(source_dir, target_dir)
    file_locations = organize_files_by_extension(target_dir)
    
    # Step 3: Create a combined report and save it to the final Excel file
    create_combined_report(df_structure, file_info, file_locations, final_excel_path, base_folder)

    print(f"Files have been copied, renamed, organized, and the combined report has been created at {final_excel_path}")

if __name__ == "__main__":
    main()


Files have been copied, renamed, organized, and the combined report has been created at CombinedFileTracker22.xlsx
